In [9]:
import pandas as pd
import numpy as np

#데이터가 3백만개 기본적으로 64비트로 가져오면 램 터진다
train_orders = pd.read_csv("data/order_products__train.csv", dtype={'order_id': np.uint32,
           'product_id': np.uint16, 'reordered': np.int8, 'add_to_cart_order': np.uint8 })
orders = pd.read_csv("data/orders.csv", dtype={'order_hour_of_day': np.uint8,
           'order_number': np.uint8, 'order_id': np.uint32, 'user_id': np.uint32,
           'order_dow': np.uint8, 'days_since_prior_order': np.float16})

In [10]:
train_orders.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


add_to_cart_order (구매 순서)
1. 일찍 산다는건 중요, 늦게 산다는 건 중요
2. 숫자가 근접하면 같이 살 확률이 높아진다

reordered (재주문)

In [11]:
orders.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [12]:
orders["user_id"].nunique()

206209

유저수 20만명정도
평균적으로 1명당 15번 구매

In [13]:
orders.head(30)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


- order_dow(dayof week): 주문 요일
- order_hour_of_day: 주문 시간대
- days_since_prior_order: 저번 주문으로부터 몇일이 지나서 샀는지

In [14]:
orders["eval_set"].value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [15]:
orders.eval_set = orders.eval_set.replace({'prior': 0, 'train': 1, 'test':2}).astype(np.uint8)

In [16]:
# 최초 구매라서 null - > 보통 한달 단위로 30
orders.days_since_prior_order = orders.days_since_prior_order.fillna(30).astype(np.uint8)

In [17]:
train_orders = train_orders.merge(orders[['user_id','order_id']], on = 'order_id', how = 'left')
train_orders.head(10)

,order_id,product_id,add_to_cart_order,reordered,user_id
0,1,49302,1,1,112108
1,1,11109,2,1,112108
2,1,10246,3,0,112108
3,1,49683,4,0,112108
4,1,43633,5,1,112108
5,1,13176,6,0,112108
6,1,47209,7,0,112108
7,1,22035,8,1,112108
8,36,39612,1,0,79431
9,36,19660,2,1,79431


reordered가 0인값은 안 중요: 한번만 샀지, 그 이후로 안 산 것 

In [18]:
train_orders = train_orders[train_orders['reordered']==1].drop('reordered',axis=1)
train_orders.head(10)

,order_id,product_id,add_to_cart_order,user_id
0,1,49302,1,112108
1,1,11109,2,112108
4,1,43633,5,112108
7,1,22035,8,112108
9,36,19660,2,79431
11,36,43086,4,79431
12,36,46620,5,79431
13,36,34497,6,79431
14,36,48679,7,79431
15,36,46979,8,79431


In [19]:
orders.set_index('order_id', drop=False, inplace=True)
orders.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
2539329,2539329,1,0,1,2,8,30
2398795,2398795,1,0,2,3,7,15
473747,473747,1,0,3,3,12,21
2254736,2254736,1,0,4,4,7,29
431534,431534,1,0,5,4,15,28
3367565,3367565,1,0,6,2,7,19
550135,550135,1,0,7,1,9,20
3108588,3108588,1,0,8,1,14,14
2295261,2295261,1,0,9,1,16,0


In [20]:
train1=orders[['order_id','eval_set']].loc[orders['eval_set']==1]
train1.head(10)

,order_id,eval_set
order_id,,
1187899,1187899,1
1492625,1492625,1
2196797,2196797,1
525192,525192,1
880375,880375,1
1094988,1094988,1
1822501,1822501,1
1827621,1827621,1
2316178,2316178,1


In [21]:
train_orders.groupby('order_id').aggregate({'product_id':lambda x: list(x)})

,product_id
order_id,
1,"[49302, 11109, 43633, 22035]"
36,"[19660, 43086, 46620, 34497, 48679, 46979]"
38,[21616]
96,"[20574, 40706, 27966, 24489, 39275]"
98,"[8859, 19731, 43654, 13176, 4357, 37664, 34065..."
112,"[27104, 21174, 5876]"
170,"[18394, 37766, 13176, 6236, 5077, 40354]"
218,"[1194, 5578]"
226,"[29883, 28427, 7754, 39947]"


In [22]:
train1['actual'] = train_orders.groupby('order_id').aggregate({'product_id':lambda x: list(x)})
train1['actual']=train1['actual'].fillna('')
train1.head(10)

,order_id,eval_set,actual
order_id,,,
1187899,1187899,1,"[196, 25133, 38928, 26405, 39657, 10258, 13032..."
1492625,1492625,1,"[22963, 7963, 16589, 32792, 41787, 22825, 2485..."
2196797,2196797,1,"[15349, 21413, 40706, 21616]"
525192,525192,1,"[47272, 37999, 13198, 43967, 40852, 17638, 298..."
880375,880375,1,"[15937, 23165, 21903, 41540]"
1094988,1094988,1,"[27555, 42347, 27596, 8834, 26604, 12075, 8467..."
1822501,1822501,1,
1827621,1827621,1,"[27435, 27086, 4210, 19934]"
2316178,2316178,1,"[32115, 28601, 15869, 37434, 3808, 15172, 8744..."


In [23]:
train1['actual'].apply(lambda x: len(x)).mean()

6.316822778925226

평균 장바구니 갯수: 6.3

In [24]:
n_actual = train1['actual'].apply(lambda x: len(x)).mean()   # 평균 카트 사이즈
test1=orders[['order_id','eval_set']].loc[orders['eval_set']==2]
test1['actual']=' '
test1.head()

,order_id,eval_set,actual
order_id,,,
2774568,2774568,2,
329954,329954,2,
1528013,1528013,2,
1376945,1376945,2,
1356845,1356845,2,


In [25]:
traintest1=pd.concat([train1,test1])
traintest1.set_index('order_id', drop=False, inplace=True)

traintest1.head(10)

,order_id,eval_set,actual
order_id,,,
1187899,1187899,1,"[196, 25133, 38928, 26405, 39657, 10258, 13032..."
1492625,1492625,1,"[22963, 7963, 16589, 32792, 41787, 22825, 2485..."
2196797,2196797,1,"[15349, 21413, 40706, 21616]"
525192,525192,1,"[47272, 37999, 13198, 43967, 40852, 17638, 298..."
880375,880375,1,"[15937, 23165, 21903, 41540]"
1094988,1094988,1,"[27555, 42347, 27596, 8834, 26604, 12075, 8467..."
1822501,1822501,1,
1827621,1827621,1,"[27435, 27086, 4210, 19934]"
2316178,2316178,1,"[32115, 28601, 15869, 37434, 3808, 15172, 8744..."


In [26]:
import gc
del orders, train1, test1
gc.collect() # 무언가를 제거할때 grabage collecter을 이용해서 램을 최적화
traintest1.tail(10)

,order_id,eval_set,actual
order_id,,,
959487,959487,2,
701105,701105,2,
64567,64567,2,
1564541,1564541,2,
309211,309211,2,
2728930,2728930,2,
350108,350108,2,
1043943,1043943,2,
2821651,2821651,2,
